In [2]:
import pandas as pd
import mysql.connector
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re


# Connect to the MySQL database
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='6833',
    database='life_expectancy'
)

cur = db.cursor()

print("connected")

connected


# Ensuring Dataset Reliability: Validation Using Python & MySQL

# 1) Average Life Expectancy Across Different Countries [ Gender wise]


In [21]:
query = """ SELECT Country, 
       Gender, 
       round(AVG(Life_Expectancy),2) AS Average_Life_Expectancy
FROM lifespan_data_clean
GROUP BY Country, Gender
ORDER BY Country, FIELD(Gender, 'Male', 'Female')
 """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns=["Country", "Gender", "Avg_Life_Expectancy"])
df

,Country,Gender,Avg_Life_Expectancy
0,Australia,Male,81.0
1,Australia,Female,85.0
2,Brazil,Male,72.0
3,Brazil,Female,79.0
4,Canada,Male,80.0
5,Canada,Female,84.0
6,France,Male,80.0
7,France,Female,86.0
8,Germany,Male,79.0
9,Germany,Female,84.0


In [ ]:
# 0   Australia    Male    81.0
# 1   Australia    Female  87.0
# 2   Brazil       Male    72.0
# 3   Brazil       Female  80.0
# 4   Canada       Male    80.0
# 5   Canada       Female  84.0
# 6   France       Male    80.0
# 7   France       Female  86.0
# 8   Germany      Male    79.0
# 9   Germany      Female  84.0
# 10  India        Male    69.0
# 11  India        Female  72.0
# 12  Japan        Male    81.0
# 13  Japan        Female  87.0
# 14  South Africa Male    61.0
# 15  South Africa Female  67.0

# https://www.worldometers.info/demographics/life-expectancy/


# 2) Women Live Longer: Comparing Male vs. Female Life Expectancy

In [8]:
query = """ select gender, round(avg(Life_Expectancy),2) from lifespan_data_clean group by gender """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns=["Gender", "Avg_Life_Expectancy"])
df

,Gender,Avg_Life_Expectancy
0,Male,75.23
1,Female,80.48


In [9]:
# The average lifespan is about 5 years longer for women than men worldwide.

# https://www.health.harvard.edu/blog/why-men-often-die-earlier-than-women-201602199137

# 3) The Link Between Weekly Exercise & Longevity

In [16]:
query = """
    SELECT 
        CASE 
            WHEN Exercise_hrs_per_week >= 2.6 THEN 'More than 2.5 hours' 
            ELSE 'Less than 2.5 hours' 
        END AS Exercise_Group,
        Gender, 
        ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
    FROM lifespan_data_clean
    GROUP BY Exercise_Group, Gender
    ORDER BY FIELD(Gender, 'Male', 'Female'), FIELD(Exercise_Group, 'More than 2.5 hours', 'Less than 2.5 hours')
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns=["Exercise Group", "Gender", "Avg_Life_Expectancy"])
df


,Exercise Group,Gender,Avg_Life_Expectancy
0,More than 2.5 hours,Male,76.58
1,Less than 2.5 hours,Male,72.80
2,More than 2.5 hours,Female,82.25
3,Less than 2.5 hours,Female,77.81


In [ ]:
# # Compared with individuals in the inactive group, those in the low-volume activity group, who exercised for an average of 92 min per week 
# had a 4 year longer life expectancy.

# https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(11)60749-6/abstract

# 4) Diet & Longevity: How Food Choices Affect Life Expectanc

In [22]:
query = """
SELECT 
    Diet_Type, 
    ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
FROM lifespan_data_clean
GROUP BY Diet_Type
ORDER BY Avg_Life_Expectancy DESC;
"""

cur.execute(query)
data = cur.fetchall()

columns = ["Diet_Type", "Avg_Life_Expectancy"]

df = pd.DataFrame(data, columns=columns)

Healthy_avg = df[df['Diet_Type'] == 'Healthy']['Avg_Life_Expectancy'].values[0]

df['Life_Expectancy_Difference'] = df['Avg_Life_Expectancy'] - Healthy_avg

df


,Diet_Type,Avg_Life_Expectancy,Life_Expectancy_Difference
0,Healthy,83.99,0.00
1,Balanced,77.45,-6.54
2,Unhealthy,71.92,-12.07


In [ ]:
# Estimated gains from Healthy dietary changes from an unhealthy diet pattern to the longevity-associated diet pattern were 10.8 years
# Estimated gains from Balanced dietary changes from an unhealthy UK diet pattern to the longevity-associated diet pattern were 6.4 years 

# https://www.nature.com/articles/s43016-023-00868-w

# 5) The Deadly Cost of Smoking: Years Lost to Tobacco

In [23]:
query = """
SELECT 
    Smoking, 
    ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
FROM lifespan_data_clean
GROUP BY Smoking
ORDER BY Avg_Life_Expectancy DESC;
"""

cur.execute(query)
data = cur.fetchall()

columns = ["Smoking", "Avg_Life_Expectancy"]

df = pd.DataFrame(data, columns=columns)

never_smoking_avg = df[df['Smoking'] == 'Never']['Avg_Life_Expectancy'].values[0]

df['Life_Expectancy_Difference'] = df['Avg_Life_Expectancy'] - never_smoking_avg

df


,Smoking,Avg_Life_Expectancy,Life_Expectancy_Difference
0,Never,84.52,0.00
1,Low,79.21,-5.31
2,Moderate,75.78,-8.74
3,High,71.59,-12.93


In [24]:
# Life expectancy decreases by 13 years on average for heavy smokers compared to people who have never smoked. 
# Moderate smokers (fewer than twenty cigarettes a day) lose an estimated 9 years, 
# while light (intermittent) smokers lose 5 years.

# https://www.cbs.nl/en-gb/news/2017/37/heavy-smokers-cut-their-lifespan-by-13-years-on-average

# 6) Drink Less, Live Longer? Alcohol Consumption & Longevity

In [25]:
query = """
SELECT 
    Alcohol_Consumption, 
    ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
FROM lifespan_data_clean
GROUP BY Alcohol_Consumption
ORDER BY Avg_Life_Expectancy DESC;
"""

cur.execute(query)
data = cur.fetchall()

columns = ["Alcohol_Consumption", "Avg_Life_Expectancy"]

df = pd.DataFrame(data, columns=columns)

never_alcohol_avg = df[df['Alcohol_Consumption'] == 'Never']['Avg_Life_Expectancy'].values[0]

df['Life_Expectancy_Difference'] = df['Avg_Life_Expectancy'] - never_alcohol_avg

df


,Alcohol_Consumption,Avg_Life_Expectancy,Life_Expectancy_Difference
0,Never,79.39,0.00
1,Low,78.76,-0.63
2,Moderate,77.85,-1.54
3,High,75.12,-4.27


In [ ]:
# Adults drinking seven to 14 drinks per week could expect, on average, a six-month shorter life expectancy
# Those drinking 14 to 25 drinks per week could expect a shorter life expectancy by one to two years
# Those drinking more than 25 drinks per week could expect a shorter life expectancy by four to five years

# https://www.health.harvard.edu/blog/sorting-out-the-health-effects-of-alcohol-2018080614427

# 7) Sleep & Longevity: How Sleep Duration Affects Life Expectancy

In [27]:
query = """
SELECT 
    CASE 
        WHEN Sleep_hrs_per_day BETWEEN 7 AND 9 THEN '7-9 Hours (Adequate Sleep)'
        ELSE 'Other'
    END AS Sleep_Group,
    ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
FROM lifespan_data_clean
GROUP BY Sleep_Group
ORDER BY Avg_Life_Expectancy DESC;
"""

cur.execute(query)
data = cur.fetchall()

columns = ["Sleep_Group", "Avg_Life_Expectancy"]

df = pd.DataFrame(data, columns=columns)

sleep_base_avg = df[df['Sleep_Group'] == '7-9 Hours (Adequate Sleep)']['Avg_Life_Expectancy'].values[0]

df['Life_Expectancy_Difference'] = df['Avg_Life_Expectancy'] - sleep_base_avg

df


,Sleep_Group,Avg_Life_Expectancy,Life_Expectancy_Difference
0,7-9 Hours (Adequate Sleep),80.70,0.00
1,Other,75.76,-4.94


In [28]:
# men who get adequate sleep live about five years longer than men who don't

# https://mcpress.mayoclinic.org/healthy-aging/how-quality-sleep-impacts-your-lifespan/


# 8) BMI & Longevity: How Weight Affects Life Expectancy

In [37]:
query = """
SELECT 
    CASE 
        WHEN BMI BETWEEN 18.5 AND 24.9 THEN 'Healthy Weight (18.5–24.9)'
        WHEN BMI BETWEEN 25.0 AND 29.9 THEN 'Overweight (25.0–29.9)'
        WHEN BMI >= 30.0 THEN 'Obese (≥30.0)'
        WHEN BMI < 18.5 THEN 'Underweight (<18.5)'
    END AS BMI_Group,
    ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
FROM lifespan_data_clean
WHERE BMI IS NOT NULL
GROUP BY BMI_Group
ORDER BY Avg_Life_Expectancy DESC;
"""

cur.execute(query)
data = cur.fetchall()

columns = ["BMI_Group", "Avg_Life_Expectancy"]

df = pd.DataFrame(data, columns=columns)

healthy_avg = df[df['BMI_Group'] == 'Healthy Weight (18.5–24.9)']['Avg_Life_Expectancy'].values[0]

df['Life_Expectancy_Difference'] = df['Avg_Life_Expectancy'] - healthy_avg

df


,BMI_Group,Avg_Life_Expectancy,Life_Expectancy_Difference
0,Healthy Weight (18.5–24.9),83.26,0.00
1,Overweight (25.0–29.9),79.54,-3.72
2,Obese (≥30.0),75.88,-7.38
3,Underweight (<18.5),74.48,-8.78


In [38]:
# Compared with individuals of healthy weight (BMI 18·5–24·9 kg/m2), life expectancy was 6·8 years shorter in obese (BMI ≥30·0 kg/m2)
# 8·3 years shorter in underweight (BMI <18·5 kg/m2) men 

# https://www.thelancet.com/journals/landia/article/PIIS2213-8587(18)30288-2/fulltext

# 9) Live, Laugh, Last Longer: How Social Life Shapes Longevity

In [42]:
query = """
WITH social_life_avg AS (
    SELECT 
        Social_Life, 
        ROUND(AVG(Life_Expectancy), 2) AS Avg_Life_Expectancy
    FROM lifespan_data_clean
    GROUP BY Social_Life
)
SELECT 
    Social_Life,
    Avg_Life_Expectancy,
    ROUND(Avg_Life_Expectancy - 
        (SELECT Avg_Life_Expectancy FROM social_life_avg WHERE Social_Life = 'High'), 2
    ) AS Life_Expectancy_Difference
FROM social_life_avg
ORDER BY FIELD(Social_Life, 'High', 'Moderate', 'Low');
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns=["Social_Life", "Avg_Life_Expectancy", "Life_Expectancy_Difference"])

df


,Social_Life,Avg_Life_Expectancy,Life_Expectancy_Difference
0,High,78.36,0.00
1,Moderate,77.83,-0.53
2,Low,77.15,-1.21


In [ ]:
# Researchers agree that social connection can help people live longer by 1-2 years and healthier lives

# https://hsph.harvard.edu/news/the-importance-of-connections-ways-to-live-a-longer-healthier-life/